In [ ]:

# 1) Install dependencies
!pip -q install streamlit pyngrok transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.2 MB/s eta 0:00:00


In [ ]:
!pip install -q --upgrade torch
!pip install -q transformers triton==3.4 kernels
!pip uninstall -q torchvision torchaudio -y


In [1]:
#login
from huggingface_hub import login
#Set your HUGGINGFACE auth token
login("your_token_here")


In [ ]:
# Colab cell 2 – clone GPT‑OSS (20 B) with Git‑LFS
!git clone https://huggingface.co/openai/gpt-oss-20b
# If you want the smaller 7 B version, replace the URL above with gpt-oss-7b
# (or whichever GGUF you have).
# The clone will download ~70 GB; use a Pro+ runtime or a persistent drive!

In [ ]:
%%writefile app.py
# app.py
import streamlit as st
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# --- Load model + tokenizer ---

@st.cache_resource
def load_model():
    model_path = "/content/gpt-oss-20b"

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(
       model_path,
       torch_dtype="auto",
       device_map="auto",   # auto = GPU if available, fallback CPU
   )
    return tokenizer, model

tokenizer, model = load_model()

# --- Streamlit UI ---
st.set_page_config(page_title="Chat with GPT OSS 20B", page_icon="🤖")
st.title("🌿 Eco‑GPT – Open‑Source Conservation Assistant")
st.caption("Ask natural‑language questions about the sensor logs you just uploaded.")
st.markdown("---")

# Input box
user_input = st.text_area("Your message:", "What is the weather like in Madrid?")

if st.button("Generate Reply"):
    if user_input.strip():
        with st.spinner("Thinking in riddles..."):
            messages = [
                {"role": "system", "content": "Always respond in riddles"},
                {"role": "user", "content": user_input},
            ]

            inputs = tokenizer.apply_chat_template(
                messages,
                add_generation_prompt=True,
                return_tensors="pt",
                return_dict=True,
            ).to(model.device)

            generated = model.generate(**inputs, max_new_tokens=200)
            output_text = tokenizer.decode(
                generated[0][inputs["input_ids"].shape[-1]:],
                skip_special_tokens=True
            )

        st.subheader("🤔 Model's answer:")
        st.write(output_text)
    else:
        st.warning("Please type a message first.")



Overwriting app.py


In [ ]:
import os, subprocess, time, threading
from pyngrok import ngrok

# (Optional) Paste your ngrok auth token here for a more stable tunnel
NGROK_AUTH_TOKEN = ""  # e.g. "2Qx...your_token...Abc"
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

PORT = 8501

# Close any existing tunnels to avoid duplicates
for t in ngrok.get_tunnels():
    try:
        ngrok.disconnect(t.public_url)
    except:
        pass

# Start Streamlit in the background
cmd = [
    "streamlit", "run", "app.py",
    "--server.port", str(PORT),
    "--server.address", "0.0.0.0",
    "--server.headless", "true",
]
log_path = "/content/streamlit.log"
log_file = open(log_path, "w")
proc = subprocess.Popen(cmd, stdout=log_file, stderr=log_file, text=True)

# Give Streamlit a moment to boot
time.sleep(3)

# Start ngrok tunnel
tunnel = ngrok.connect(addr=PORT, proto="http")
public_url = tunnel.public_url
print("Your app is live at:", public_url)

# Live-tail the Streamlit logs so you can see when it's ready
def tail_logs(path):
    with open(path, "r") as f:
        f.seek(0, os.SEEK_END)
        while True:
            line = f.readline()
            if line:
                print(line, end="")
            else:
                time.sleep(0.5)

threading.Thread(target=tail_logs, args=(log_path,), daemon=True).start()

# Keep the cell alive so the tunnel stays open
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    pass


Your app is live at: https://d56916a9a3d0.ngrok-free.app
2025-09-02 16:14:55.973607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756829696.002302   36956 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756829696.012785   36956 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756829696.035533   36956 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756829696.035602   36956 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756829696.035610 

In [ ]:
%%writefile sample_logs.csv
timestamp,sensor_id,event_type,payload,metadata
2024-08-01T06:14:00,cam_trap_01,image,Jaguar observed near stream #3,"{"lat": -3.45, "lon": -62.78}"
2024-08-01T06:15:00,mic_01,audio,Cicada chorus intensity: high,"{"temp_c": 27.3}"
2024-08-01T06:20:00,env_01,temperature,Air temperature 28°C,"{"humidity":71}"


Writing sample_logs.csv
